Carga de datos simulados en BBDD

In [6]:
import pandas as pd
from faker import Faker
import numpy as np
import random
from datetime import timedelta, datetime
import requests
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import random


def generar_telefono_es():
    return '34' + str(random.randint(600000000, 699999999))


# Feriados de la Comunidad de Madrid en 2025
feriados_madrid_2025 = [
    "2024-12-25",
    "2025-01-01",
    "2025-01-06",
    "2025-04-17",
    "2025-04-18",
    "2025-05-01",
    "2025-05-02",
    "2025-05-15",
    "2025-07-25",
    "2025-08-15",
    "2025-11-01",
    "2025-11-10",
    "2025-12-06",
    "2025-12-08",
    "2025-12-25",
]

# Convertimos a objetos datetime.date para uso posterior
feriados_madrid_2025 = [datetime.strptime(d, "%Y-%m-%d").date() for d in feriados_madrid_2025]


# --- CARGAR VARIABLES DE ENTORNO PARA LA CONEXIÓN A LA BBDD ---
load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

# --- PASO 1: GENERAR 5000 RESERVAS SIMULADAS ---
print("Generando reservas simuladas...")

n_reservas = 15000
fake = Faker('es_ES')
np.random.seed(42)
random.seed(42)

id_instalaciones = list(range(1, 14))
horas_disponibles = list(range(7, 23))

# Asignamos más peso a las horas pico (18 a 21)
pesos_horarios = [1 if h < 18 or h > 21 else 5 for h in horas_disponibles]

reservas = []
for i in range(n_reservas):
    days_ago = random.randint(0, 365)
    start_hour = random.choices(horas_disponibles, weights=pesos_horarios, k=1)[0] # Las horas pico tienen cinco veces más peso que una hora no pico dentro del cálculo de la probabilidad relativa.
    dt_inicio = fake.date_time_between(start_date=f'-{days_ago}d', end_date='now').replace(
        hour=start_hour, minute=0, second=0, microsecond=0
    )

    scale_exponencial = 3.0
    max_dias_antelacion = 15
    dias_antelacion_float = np.random.exponential(scale=scale_exponencial)
    creation_delta_days = min(round(dias_antelacion_float), max_dias_antelacion)
    dt_creacion = dt_inicio - timedelta(days=int(creation_delta_days))

    reservas_previas = np.random.poisson(2)
    cancelaciones_previas = np.random.binomial(reservas_previas, 0.2) if reservas_previas > 0 else 0

    reserva = {
        'id_reserva': i + 10000,
        'id_instalacion': random.choice(id_instalaciones),
        'ds_nombre_cliente': fake.name(),
        'dt_fechahora_inicio': dt_inicio,
        'dt_fechahora_fin': dt_inicio + timedelta(hours=1),
        'dt_fechahora_creacion': dt_creacion,
        'antelacion_dias': creation_delta_days,
        'reservas_previas': reservas_previas,
        'cancelaciones_previas': cancelaciones_previas,
        'es_finde': int(dt_inicio.weekday() >= 5),
        'es_horario_pico': int(start_hour in range(18, 22)),
        'es_feriado': int(dt_inicio.date() in feriados_madrid_2025)
    }
    reservas.append(reserva)

df_reservas = pd.DataFrame(reservas)

# --- PASO 2: CONSULTAR LLUVIA DESDE OPEN-METEO ---
print("Consultando lluvia histórica desde Open-Meteo...")

latitud_madrid = 40.4165
longitud_madrid = -3.7026
fechas_unicas = df_reservas['dt_fechahora_inicio'].dt.normalize().unique()
fecha_inicio_str = min(fechas_unicas).strftime('%Y-%m-%d')
fecha_fin_str = max(fechas_unicas).strftime('%Y-%m-%d')

url_api = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitud_madrid,
    "longitude": longitud_madrid,
    "start_date": fecha_inicio_str,
    "end_date": fecha_fin_str,
    "daily": "precipitation_sum",
    "timezone": "Europe/Berlin"
}

datos_lluvia = {}
try:
    response = requests.get(url_api, params=params)
    response.raise_for_status()
    data_api = response.json()
    for i, fecha_str in enumerate(data_api['daily']['time']):
        fecha = pd.to_datetime(fecha_str).normalize()
        precipitacion = data_api['daily']['precipitation_sum'][i]
        datos_lluvia[fecha] = 1 if precipitacion is not None and precipitacion > 1.0 else 0
    print("Lluvia consultada correctamente.")
except requests.exceptions.RequestException as e:
    print(f"Error consultando lluvia: {e}. Se asumirá sin lluvia.")
    for fecha in fechas_unicas:
        datos_lluvia[pd.to_datetime(fecha).normalize()] = 0

df_reservas['lluvia'] = df_reservas['dt_fechahora_inicio'].dt.normalize().map(datos_lluvia).fillna(0).astype(bool)

# --- PASO 3: CALCULAR PROBABILIDAD DE CANCELACIÓN ---
print("Calculando probabilidad de cancelación...")

base = 0.10
efecto_finde = 0.05
efecto_feriado = 0.10
efecto_horario_pico = 0.10
efecto_cancelaciones_previas = 0.02
efecto_reservas_previas = -0.01
efecto_lluvia = 0.35

df_reservas['nu_prob_cancelacion'] = (
    base
    + efecto_finde * df_reservas['es_finde']
    + efecto_feriado * df_reservas['es_feriado']
    + efecto_horario_pico * df_reservas['es_horario_pico']
    + efecto_cancelaciones_previas * df_reservas['cancelaciones_previas']
    + efecto_reservas_previas * df_reservas['reservas_previas']
    + efecto_lluvia * df_reservas['lluvia']
).clip(0, 1)

df_reservas['nu_prob_cancelacion'] = df_reservas['nu_prob_cancelacion'].astype(float)

df_reservas.rename(columns={'nu_prob_cancelacion': 'probabilidad_cancelacion'}, inplace=True)

df_reservas['es_cancelada'] = (np.random.rand(len(df_reservas)) < df_reservas['probabilidad_cancelacion']).astype(int) # simula si cada reserva fue cancelada o no en base a su probabilidad de cancelación.

df_reservas['ds_estado'] = np.where(df_reservas['es_cancelada'] == 1, 'Cancelada', 'Confirmada')
df_reservas['ds_comentarios'] = ''
df_reservas['ds_telefono'] = [generar_telefono_es() for _ in range(len(df_reservas))]
df_reservas['es_simulado'] = True

# --- PASO 4: GUARDAR EN LA BASE DE DATOS ---
print("Guardando reservas en la base de datos...")

columnas_tabla = [
    'id_reserva',
    'id_instalacion',
    'ds_nombre_cliente',
    'dt_fechahora_inicio',
    'dt_fechahora_fin',
    'dt_fechahora_creacion',
    'antelacion_dias',
    'reservas_previas',
    'cancelaciones_previas',
    'es_finde',
    'es_horario_pico',
    'es_feriado',
    'ds_estado',
    'ds_comentarios',
    'ds_telefono',
    'lluvia',
    'es_simulado',
    'probabilidad_cancelacion'
]

df_para_insertar = df_reservas[columnas_tabla].copy()

try:
    df_para_insertar.to_sql(
        'reservas',
        engine,
        if_exists='append',
        index=False
    )
    print(f"Éxito: se insertaron {len(df_para_insertar)} reservas simuladas.")
except Exception as e:
    print(f"Error insertando en la base de datos: {e}")


Generando reservas simuladas...
Consultando lluvia histórica desde Open-Meteo...
Lluvia consultada correctamente.
Calculando probabilidad de cancelación...
Guardando reservas en la base de datos...
Éxito: se insertaron 15000 reservas simuladas.
